In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

C:\Users\Roze\AppData\Local\Temp\ipykernel_11916\4086620180.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_clean = pd.read_csv("selected_users_data.csv", encoding="utf8")
anime_data = pd.read_csv("anime-dataset-2023.csv", encoding="utf8")

In [3]:
data_clean.head()

,user_id,Username,anime_id,Anime Title,rating
0,436,Katsuki,6408,"""Bungaku Shoujo"" Movie",8
1,436,Katsuki,48,.hack//Sign,8
2,436,Katsuki,1487,.hack//Unison,6
3,436,Katsuki,6682,11eyes,6
4,436,Katsuki,32601,12-sai.: Chicchana Mune no Tokimeki,7


In [4]:
print(data_clean[data_clean['anime_id']  == 35790])

         user_id       Username  anime_id                  Anime Title  rating
1572         436        Katsuki     35790  Tate no Yuusha no Nariagari       3
4498        1475       otaku515     35790  Tate no Yuusha no Nariagari       8
7507        2152       leapylee     35790  Tate no Yuusha no Nariagari       8
10309       2276          Sitha     35790  Tate no Yuusha no Nariagari       8
14649       3380  Tracer_Senpai     35790  Tate no Yuusha no Nariagari       8
...          ...            ...       ...                          ...     ...
1482270  1281059      BakaKaito     35790  Tate no Yuusha no Nariagari      10
1483330  1281765    RaphaelRobo     35790  Tate no Yuusha no Nariagari       5
1484996  1283219       Spade_X3     35790  Tate no Yuusha no Nariagari       9
1486026  1286093       JadeRuby     35790  Tate no Yuusha no Nariagari       9
1488813  1288175           GPJW     35790  Tate no Yuusha no Nariagari       8

[659 rows x 5 columns]


In [5]:
userRatings = data_clean.pivot_table(index=['user_id'], columns=['anime_id'], values='rating')

In [6]:
userRatings.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,55591,55610,55616,55640,55647,55651,55676,55720,55818,55878
user_id,,,,,,,,,,,,,,,,,,,,,
436,10.0,8.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105,NaN,NaN,NaN,8.0,8.0,9.0,8.0,8.0,8.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1475,8.0,8.0,9.0,8.0,6.0,NaN,7.0,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1645,9.0,9.0,7.0,8.0,NaN,2.0,9.0,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2152,9.0,NaN,7.0,6.0,NaN,7.0,9.0,8.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
userRatings.shape

(1351, 14994)

In [7]:
# corrMatrix = userRatings.corr()
# corrMatrix.head()

In [8]:
corrMatrix = userRatings.corr(method='pearson', min_periods=100)
corrMatrix.head()

224820036

In [14]:
corrMatrix.size
corrMatrix.shape

(14994, 14994)

In [9]:
# myRatings = userRatings.loc[9522].dropna()
# data_dict = {14813: 10, 37779: 10, 37430: 10, 2566: 2}
# data_dict = {17389: 8, 35790: 8, 1735: 9, 16498: 9}

new_ratings = []
for _ in range(5):  # Assume the user rates 5 animes (adjust as needed)
    anime_id = input("Enter Anime ID: ")
    rating = float(input("Enter Rating (between 1 and 10): "))
    new_ratings.append((int(anime_id), rating))

print(new_ratings)
print(dict(new_ratings))
myRatings = pd.Series(dict(new_ratings))
print(myRatings)
myRatings

[(17389, 8.0), (35790, 8.0), (1735, 9.0), (16498, 9.0), (14813, 5.0)]
{17389: 8.0, 35790: 8.0, 1735: 9.0, 16498: 9.0, 14813: 5.0}
17389    8.0
35790    8.0
1735     9.0
16498    9.0
14813    5.0
dtype: float64


17389    8.0
35790    8.0
1735     9.0
16498    9.0
14813    5.0
dtype: float64

In [10]:
anime_id_to_check = 2566

if anime_id_to_check in corrMatrix.columns:
    print(f"Anime ID {anime_id_to_check} is present in the correlation matrix.")
else:
    print(f"Anime ID {anime_id_to_check} is not present in the correlation matrix.")


Anime ID 2566 is present in the correlation matrix.


In [43]:
simCandidates = pd.Series()
for i in range(1, len(myRatings.index)):
    # print("Adding sims for " + str(myRatings.index[i]) + "...")
    sims = corrMatrix[myRatings.index[i]].dropna()
    # sims = sims.map(lambda x: x * myRatings[i])
    sims = sims.map(lambda x: x * myRatings.loc[myRatings.index[i]])
    # simCandidates = simCandidates.append(sims)
    simCandidates = pd.concat([simCandidates, sims])
    
print("sorting...")
simCandidates.sort_values(inplace= True, ascending= False)

user_input_anime_ids = [anime_id for anime_id, _ in new_ratings]
simCandidates = simCandidates[~simCandidates.index.isin(user_input_anime_ids)]
top_5_results = simCandidates.head(10)
anime_df = top_5_results.reset_index(name='weight_score')

# Merge with the anime_data DataFrame to retrieve title and score
result_df = pd.merge(anime_df, anime_data, how='left', left_on='index', right_on='anime_id')
print(result_df)
result_df = result_df[['anime_id', 'Name', 'Score']]

# Print the resulting DataFrame
result_df

sorting...
   index  weight_score  anime_id                                     Name  \
0     20      6.469236        20                                   Naruto   
1    269      6.149403       269                                   Bleach   
2  25777      6.050869     25777              Shingeki no Kyojin Season 2   
3  18397      5.800070     18397                   Shingeki no Kyojin OVA   
4  25781      5.731697     25781      Shingeki no Kyojin: Kuinaki Sentaku   
5   1669      5.662688      1669              Bakuten Shoot Beyblade 2002   
6   6702      5.618733      6702                               Fairy Tail   
7  18045      5.606780     18045  Koi to Senkyo to Chocolate: Koi Imouto!   
8  38524      5.519468     38524       Shingeki no Kyojin Season 3 Part 2   
9  51462      5.510894     51462                     Isekai Nonbiri Nouka   

                      English name             Other name Score  \
0                           Naruto                    ナルト  7.99   
1      

C:\Users\Roze\AppData\Local\Temp\ipykernel_11916\1540976389.py:8: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  simCandidates = pd.concat([simCandidates, sims])


,anime_id,Name,Score
0,20,Naruto,7.99
1,269,Bleach,7.92
2,25777,Shingeki no Kyojin Season 2,8.5
3,18397,Shingeki no Kyojin OVA,7.87
4,25781,Shingeki no Kyojin: Kuinaki Sentaku,8.41
5,1669,Bakuten Shoot Beyblade 2002,6.7
6,6702,Fairy Tail,7.57
7,18045,Koi to Senkyo to Chocolate: Koi Imouto!,6.83
8,38524,Shingeki no Kyojin Season 3 Part 2,9.05
9,51462,Isekai Nonbiri Nouka,7.58


In [44]:
simCandidates = pd.Series()
for i in range(1, len(myRatings.index)):
    # print("Adding sims for " + str(myRatings.index[i]) + "...")
    sims = corrMatrix[myRatings.index[i]].dropna()
    # sims = sims.map(lambda x: x * myRatings[i])
    sims = sims.map(lambda x: x * myRatings.loc[myRatings.index[i]])
    # simCandidates = simCandidates.append(sims)
    simCandidates = pd.concat([simCandidates, sims])
    
print(simCandidates.size)
# Check if there are any duplicates
has_duplicates = simCandidates.duplicated().any()

if has_duplicates:
    duplicate_values = simCandidates[simCandidates.duplicated()]
    print("Duplicate values:", duplicate_values)

simCandidates = simCandidates.groupby(simCandidates.index).sum()
simCandidates = simCandidates.drop(myRatings.index)

print("sorting...")
simCandidates.sort_values(inplace= True, ascending= False)

# user_input_anime_ids = [anime_id for anime_id, _ in new_ratings]
# simCandidates = simCandidates[~simCandidates.index.isin(user_input_anime_ids)]
top_5_results = simCandidates.head(10)
anime_df = top_5_results.reset_index(name='weight_score')

# Merge with the anime_data DataFrame to retrieve title and score
result_df = pd.merge(anime_df, anime_data, how='left', left_on='index', right_on='anime_id')
print(result_df)
result_df = result_df[['anime_id', 'Name', 'Score']]

# Print the resulting DataFrame
result_df

12285
Duplicate values: 1735     3.662376
16498    9.000000
dtype: float64
sorting...
   index  weight_score  anime_id  \
0  18045     16.830803     18045   
1  11757     15.871798     11757   
2   9919     15.677083      9919   
3  36896     15.600579     36896   
4  31553     15.528720     31553   
5    269     15.429736       269   
6  22043     15.318518     22043   
7  26243     15.172410     26243   
8   6702     15.128791      6702   
9  21881     15.105696     21881   

                                                Name  \
0            Koi to Senkyo to Chocolate: Koi Imouto!   
1                                   Sword Art Online   
2                                     Ao no Exorcist   
3  Boku no Hero Academia the Movie 1: Futari no Hero   
4                        Charlotte: Tsuyoimono-tachi   
5                                             Bleach   
6                                  Fairy Tail (2014)   
7                                    Owari no Seraph   
8            

C:\Users\Roze\AppData\Local\Temp\ipykernel_11916\3938410546.py:8: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  simCandidates = pd.concat([simCandidates, sims])


,anime_id,Name,Score
0,18045,Koi to Senkyo to Chocolate: Koi Imouto!,6.83
1,11757,Sword Art Online,7.2
2,9919,Ao no Exorcist,7.5
3,36896,Boku no Hero Academia the Movie 1: Futari no Hero,7.56
4,31553,Charlotte: Tsuyoimono-tachi,7.49
5,269,Bleach,7.92
6,22043,Fairy Tail (2014),7.65
7,26243,Owari no Seraph,7.49
8,6702,Fairy Tail,7.57
9,21881,Sword Art Online II,6.7


In [35]:
anime_id_to_check = 51462

# if anime_id_to_check in simCandidates.index:
#     print(f"Data with anime_id {anime_id_to_check} is present in simCandidates.")
# else:
#     print(f"Data with anime_id {anime_id_to_check} is not present in simCandidates.")

found = False
for position, anime_id in enumerate(simCandidates.index, start=1):
    if anime_id == anime_id_to_check:
        found = True
        break

if found:
    print(f"Anime ID {anime_id_to_check} is ranked #{position} in simCandidates.")
else:
    print(f"Anime ID {anime_id_to_check} is not present in simCandidates.")



Anime ID 51462 is ranked #10 in simCandidates.


In [31]:
index_to_check = 14000

try:
    anime_id_at_index = simCandidates.index[index_to_check - 1]
    print(f"The anime ID at index {index_to_check} in simCandidates is {anime_id_at_index}.")
except IndexError:
    print(f"Index {index_to_check} is out of bounds for simCandidates.")


Index 14000 is out of bounds for simCandidates.
